In [8]:
import requests
import mysql.connector
from mysql.connector import Error

In [ ]:

#Substitua pela sua chave de API
api_key = 'c90c917f98d91bd0c1fd096a6b77ab58'

#URL para acessar a API do TMDB
url = f"https://api.themoviedb.org/3/movie/popular?api_key={api_key}&language=pt-BR&page=1"

#Fazendo a requisição à API
response = requests.get(url)

#Verificando se a requisição foi bem-sucedida (código 200)
if response.status_code == 200:
    data = response.json()
    
    #Verificando se o campo 'results' está presente nos dados
    if 'results' in data:        
        for movie in data['results']:
            titulo = movie.get('title', 'Título não disponível')
            data_lancamento = movie.get('release_date', 'Data de lançamento não disponível')
            popularidade = movie.get('popularity', 'Popularidade não disponível')
            sinopse = movie.get('overview', 'Sinopse não disponível')
            nota = movie.get('vote_average', 'Nota não disponível')
            
            print(f"Título: {titulo}")
            print(f"Data de lançamento: {data_lancamento}")
            print(f"Popularidade: {popularidade}")
            print(f"Sinopse: {sinopse}")
            print(f"Nota: {nota}")
            print('---')
    else:
        print("Erro: O campo 'results' não foi encontrado na resposta da API.")
else:
    print(f"Erro: Não foi possível acessar a API. Código de status: {response.status_code}")


In [ ]:
# Função para conectar ao MySQL
def connect_to_mysql():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            database='dataset_TMDB',
            user='host',
            password='Are@0051'
        )
        if connection.is_connected():
            print("Conectado ao MySQL")
            return connection
    except Error as e:
        print(f"Erro ao conectar ao MySQL: {e}")
        return None

In [ ]:


# Função para inserir dados no MySQL
def insert_movie_data(connection, movie):
    try:
        cursor = connection.cursor()
        insert_query = """
        INSERT INTO filmes_populares_TMDB (titulo, data_lancamento, popularidade, sinopse, nota)
        VALUES (%s, %s, %s, %s, %s)
        """
        data_tuple = (movie['title'], movie['release_date'], movie['popularity'], movie['overview'], movie['vote_average'])
        cursor.execute(insert_query, data_tuple)
        connection.commit()
        print(f"Filme '{movie['title']}' inserido com sucesso!")
    except Error as e:
        print(f"Erro ao inserir dados no MySQL: {e}")

# Substitua pela sua chave de API
api_key = 'c90c917f98d91bd0c1fd096a6b77ab58'

# URL para acessar a API do TMDB
url = f"https://api.themoviedb.org/3/movie/popular?api_key={api_key}&language=pt-BR&page=1"

# Fazendo a requisição à API
response = requests.get(url)
data = response.json()

# Conectar ao banco de dados
connection = connect_to_mysql()

# Inserir dados no MySQL
if connection:
    for movie in data['results']:
        insert_movie_data(connection, movie)

    # Fechar a conexão
    connection.close()
